In [2]:
# PROYECTO DE INTELIGENCIA DE DATOS
#Propietario: Zúñiga Zavala Víctor Hugo
#Aplicación a BRADESCO.

import os
import glob
import pandas as pd

#Cargado optimizado de datos

print("Se comenzaran a cargar los datos")

# Limpieza del entorno.

if os.path.exists('/content/COVID-19'):
    !rm -rf /content/COVID-19

# Configuración Manual por Git para descargar sólo la data necesaria

#git; distribuidor de cambios

!mkdir /content/COVID-19
%cd /content/COVID-19
!git init
!git remote add origin https://github.com/CSSEGISandData/COVID-19.git
!git config core.sparseCheckout true
!echo "csse_covid_19_data/csse_covid_19_daily_reports_us/" >> .git/info/sparse-checkout

print("Descargando archivos")

!git pull --depth=1 origin master

%cd /content


print("Descarga completada")

#Transformación y limpieaza

#Definimos una variable string de la ub de los archivos
path_repo = '/content/COVID-19/csse_covid_19_data/csse_covid_19_daily_reports_us'

all_files = glob.glob(os.path.join(path_repo, "*.csv"))
#motor de busqueda,constructor inteligente
print(f" Procesando {len(all_files)} archivos csv")

#definimos variable limpia y vacia para la tabla historica.
dataframes_temp = []

for filename in all_files:
    # Leer archivo
    df = pd.read_csv(filename, index_col=None, header=0)

    #  Estandarización de nombres de columnas (Quitar espacios, mayúsculas)
    df.columns = [c.strip().lower().replace('/', '_').replace(' ', '_') for c in df.columns]

    #  Homologación de columna Estado
    # Algunos archivos viejos usan 'state', otros 'province_state'. Unificamos sin duplicados.
    if 'state' in df.columns:
        df.rename(columns={'state': 'province_state'}, inplace=True)

    # Obtención de Fecha desde el nombre del archivo (MM-DD-YYYY.csv)
    nombre_clean = os.path.basename(filename).replace(".csv", "")
    try:
        df['fecha_corte'] = pd.to_datetime(nombre_clean, format='%m-%d-%Y')
        dataframes_temp.append(df)
    except ValueError:
        continue # Si hay un archivo basura, lo saltamos

#  (Unir los 1062+ datafram<es)
if len(dataframes_temp) > 0:
    df_master = pd.concat(dataframes_temp, axis=0, ignore_index=True)

    # Ordenamiento Lógico (Por Estado y Fecha)
    if 'province_state' in df_master.columns:
        df_master = df_master.sort_values(by=['province_state', 'fecha_corte'])
    else:
        df_master = df_master.sort_values(by=['fecha_corte'])

    # Calidad de Datos (Rellenar Nulos en métricas con 0)
    cols_metricas = ['confirmed', 'deaths', 'recovered', 'active', 'people_tested', 'people_hospitalized']
    for col in cols_metricas:
        if col in df_master.columns:
            df_master[col] = df_master[col].fillna(0)

   #checks finales
    print("REPORTE DE CALIDAD")
    print(f"Dimensiones finales: {df_master.shape[0]:,} filas x {df_master.shape[1]} columnas")
    print(f"Rango de Fechas: {df_master['fecha_corte'].min().date()} al {df_master['fecha_corte'].max().date()}")
    print("\nPrimeras 5 filas:")
    display(df_master.head())

else:
    print("No se encontraron archivos válidos para procesar.")

Se comenzaran a cargar los datos
/content/COVID-19
hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/COVID-19/.git/
Descargando archivos
remote: Enumerating objects: 2441, done.
remote: Counting objects: 100% (2441/2441), done.
remote: Compressing objects: 100% (2361/2361), done.
remote: Total 2441 (delta 138), reused 248 (delta 78), pack-reused 0 (from 0)
Receiving objects: 100% (2441/2441), 354.47 MiB | 11.67 MiB/s, done.
Resolving deltas: 100% (138/138), done.
From https://github.com/CSSEGISandData/COVID-19
 * b

,province_state,country_region,last_update,lat,long_,confirmed,deaths,recovered,active,fips,...,people_hospitalized,case_fatality_ratio,uid,iso3,testing_rate,hospitalization_rate,date,people_tested,mortality_rate,fecha_corte
580,Alabama,US,2020-04-12 23:18:15,32.3182,-86.9023,3667,93,0.0,0.0,1.0,...,437.0,NaN,84000001.0,USA,NaN,12.264945,2020-04-12,62029.0,2.536133,2020-04-12
54997,Alabama,US,2020-04-13 23:07:54,32.3182,-86.9023,3870,99,0.0,0.0,1.0,...,457.0,NaN,84000001.0,USA,NaN,12.238886,2020-04-13,64636.0,2.558140,2020-04-13
60163,Alabama,US,2020-04-14 23:33:31,32.3182,-86.9023,4041,114,0.0,0.0,1.0,...,493.0,NaN,84000001.0,USA,NaN,12.471541,2020-04-14,67557.0,2.821084,2020-04-14
31443,Alabama,US,2020-04-15 22:56:51,32.3182,-86.9023,4307,118,0.0,0.0,1.0,...,525.0,NaN,84000001.0,USA,NaN,12.883436,2020-04-15,70823.0,2.739726,2020-04-15
55172,Alabama,US,2020-04-16 23:30:51,32.3182,-86.9023,4465,133,0.0,0.0,1.0,...,553.0,NaN,84000001.0,USA,NaN,12.727273,2020-04-16,73878.0,2.978723,2020-04-16


In [3]:
#VARIABLES

print("Generando variables de inteligencia")

# AGRUPACIÓN ESTRATÉGICA (Nivel Estado)
# Para analizar tendencias, ocupamos un mayor volumen.
# Agrupamos por Estado y Fecha para tener series de tiempo sólidas.
df_state = df_master.groupby(['province_state', 'fecha_corte'])[[
    'confirmed', 'deaths', 'recovered', 'people_tested', 'people_hospitalized'
]].sum().reset_index()

# Ordenamos para asegurar que los cálculos de diferencias sean cronológicos
df_state = df_state.sort_values(by=['province_state', 'fecha_corte'])


# VARIABLES TEMPORALES y deltas básicas


# Variables de Calendario (para filtros en Power BI)
df_state['anio'] = df_state['fecha_corte'].dt.year
df_state['mes'] = df_state['fecha_corte'].dt.month
df_state['mes_nombre'] = df_state['fecha_corte'].dt.strftime('%B')
df_state['semana_anio'] = df_state['fecha_corte'].dt.isocalendar().week

# CÁLCULO DE VARIACIONES DIARIAS (Des-acumular)
# Usamos groupby + diff para restar el valor de hoy menos el de ayer por estado
df_state['nuevos_casos'] = df_state.groupby('province_state')['confirmed'].diff().fillna(0)
df_state['nuevas_muertes'] = df_state.groupby('province_state')['deaths'].diff().fillna(0)

# Limpieza de "Errores de Captura" (Valores negativos)

# Para el Dashboard, convertimos negativos a 0 para no romper las gráficas.
df_state['nuevos_casos'] = df_state['nuevos_casos'].clip(lower=0)
df_state['nuevas_muertes'] = df_state['nuevas_muertes'].clip(lower=0)

#Vaeiables complejas

# SUAVIZADO DE TENDENCIAS (Medias Móviles de 7 días)
# Esto elimina el "efecto fin de semana" (cuando no reportan datos) y muestra la tendencia real.
# tomando estrategias tipo gubernamentales.
df_state['casos_promedio_7d'] = df_state.groupby('province_state')['nuevos_casos'].transform(lambda x: x.rolling(window=7).mean()).fillna(0)
df_state['muertes_promedio_7d'] = df_state.groupby('province_state')['nuevas_muertes'].transform(lambda x: x.rolling(window=7).mean()).fillna(0)

# VARIABLES RELATIVAS (KPIs de Impacto)
# Tasa de Letalidad Acumulada (%): ¿De los contagiados, cuántos mueren?
df_state['tasa_letalidad_pct'] = (df_state['deaths'] / df_state['confirmed']) * 100
df_state['tasa_letalidad_pct'] = df_state['tasa_letalidad_pct'].fillna(0) # Div by zero fix

# ELOCIDAD DE LA PANDEMIA (Growth Factor)
# Insight : Si el factor es > 1, la pandemia crece exponencialmente. Si es < 1, decae.
# Formula proxy: Nuevos Casos Hoy / Nuevos Casos Ayer (suavizados)
df_state['factor_crecimiento'] = df_state['casos_promedio_7d'] / df_state.groupby('province_state')['casos_promedio_7d'].shift(1)
df_state['factor_crecimiento'] = df_state['factor_crecimiento'].fillna(0).replace([float('inf'), -float('inf')], 0)

# SEMÁFORO DE EVOLUCIÓN (Variable Categórica para BI para matriz)
# Clasificamos el día según la tendencia de crecimiento
def clasificar_tendencia(factor):
    if factor > 1.2: return "Crecimiento Exponencial "
    elif factor > 1.0: return "Crecimiento Sostenido "
    elif factor > 0.8: return "Estabilización "
    else: return "Declive "

df_state['estatus_pandemia'] = df_state['factor_crecimiento'].apply(clasificar_tendencia)


# EXPORTACIÓN FINAL (doc para power BI)

print("\n VARIABLES CREADAS EXITOSAMENTE")
print("Ejemplo de Nuevas Variables (NY recientes):")
cols_demo = ['fecha_corte', 'nuevos_casos', 'casos_promedio_7d', 'factor_crecimiento', 'estatus_pandemia']
display(df_state[df_state['province_state'] == 'New York'].tail(5)[cols_demo])

# Guardado automatico en entorno.
df_state.to_csv('covid_us_intelligence_final.csv', index=False)
print("\n Archivo 'covid_us_intelligence_final.csv' listo para descargar.")

Generando variables de inteligencia

 VARIABLES CREADAS EXITOSAMENTE
Ejemplo de Nuevas Variables (NY recientes):


,fecha_corte,nuevos_casos,casos_promedio_7d,factor_crecimiento,estatus_pandemia
39289,2023-03-05,748.0,1617.285714,0.985721,Estabilización
39290,2023-03-06,2395.0,1538.285714,0.951153,Estabilización
39291,2023-03-07,1026.0,1377.285714,0.895338,Estabilización
39292,2023-03-08,1340.0,1508.285714,1.095115,Crecimiento Sostenido
39293,2023-03-09,1368.0,1367.714286,0.906801,Estabilización



 Archivo 'covid_us_intelligence_final.csv' listo para descargar.
